In [1]:
from create_profiles import load_data
import random
import operator

def print_profile(profile):
    for country, ballot in profile.items():
        print(f"{country}: {ballot}")

In [2]:
profiles_per_edition = load_data()

In [33]:
def get_all_alternatives(profile):
    return set(list(profile.values())[0]).union(set(list(profile.values())[1]))

def random_dictatorship(profile):
    chosen_agent = random.choice(list(profile.keys()))
    return profile[chosen_agent][0]

def stv_with_random_dictatorship(profile):
    remaining_countries = get_all_alternatives(profile)

    while(len(remaining_countries) > 1):
        chosen_agent = random.choice(list(profile.keys()))
        for country in reversed(profile[chosen_agent]):
            if country in remaining_countries:
                remaining_countries.remove(country)
                break
    
    return remaining_countries.pop()

def scoring_social_choice_function(profile, scoring_system):
    points_hist = {country: 0 for country in get_all_alternatives(profile)}
    for ballot in profile.values():
        for to_country, points in zip(ballot, scoring_system):
            points_hist[to_country] += points
    return max(points_hist.items(), key=operator.itemgetter(1))[0]

def song_festival_rules(profile):
    scoring_system = [12, 10, 8, 7, 6, 5, 4, 3, 2, 1]
    return scoring_social_choice_function(profile, scoring_system)

def borda(profile):
    scoring_system = list(reversed(range(len(profile) - 1)))
    return scoring_social_choice_function(profile, scoring_system)

def plurality(profile):
    scoring_system = [1]
    return scoring_social_choice_function(profile, scoring_system)

all_scfs = [random_dictatorship, stv_with_random_dictatorship, plurality, borda, song_festival_rules]
all_scf_names = [scf.__name__ for scf in all_scfs]
all_scfs_with_names = list(zip(all_scfs, all_scf_names))

In [34]:
profile = profiles_per_edition['1975f']

print_profile(profiles_per_edition['1975f'])

Belgium: ['Ireland', 'United Kingdom', 'Switzerland', 'Malta', 'Italy', 'Yugoslavia', 'Spain', 'The Netherlands', 'France', 'Israel', 'Finland', 'Germany', 'Luxembourg', 'Monaco', 'Norway', 'Portugal', 'Sweden', 'Turkey']
Finland: ['Italy', 'The Netherlands', 'Spain', 'United Kingdom', 'Sweden', 'Luxembourg', 'Switzerland', 'Israel', 'Monaco', 'Ireland', 'Belgium', 'France', 'Germany', 'Malta', 'Norway', 'Portugal', 'Turkey', 'Yugoslavia']
France: ['United Kingdom', 'Switzerland', 'Malta', 'Sweden', 'Ireland', 'The Netherlands', 'Italy', 'Luxembourg', 'Portugal', 'Israel', 'Belgium', 'Finland', 'Germany', 'Monaco', 'Norway', 'Spain', 'Turkey', 'Yugoslavia']
Germany: ['Finland', 'United Kingdom', 'The Netherlands', 'Belgium', 'Switzerland', 'Spain', 'Italy', 'Monaco', 'Yugoslavia', 'Israel', 'France', 'Ireland', 'Luxembourg', 'Malta', 'Norway', 'Portugal', 'Sweden', 'Turkey']
Ireland: ['France', 'Luxembourg', 'The Netherlands', 'Spain', 'Italy', 'Finland', 'Yugoslavia', 'United Kingdom'

In [39]:
def add_linear_cost(profile):
    """
    profile: a dict representing the profile of the form {agent: [choice_1, ... , choice_n]}
    returns: a dict of the form: {agent: [(choice_1, 0), ... , (choice_n, n-1)]}
    """
    return {agent: list(zip(ballot, range(len(ballot)))) for agent, ballot in profile.items()}

def get_total_cost_for_alternative(alternative, profile_with_costs):
        cost = 0
        for agent, ballot_with_costs in profile_with_costs.items():
            alternative_cost_dict = dict(ballot_with_costs)
            if alternative in alternative_cost_dict:
                cost += alternative_cost_dict[alternative]
        return cost

def get_lowest_cost(alternatives, profile_with_costs):
    lowest_cost = 99999
    best_a = None
    for a in alternatives:
        cost = get_total_cost_for_alternative(a, profile_with_costs)
        if cost < lowest_cost:
            best_a = a
            lowest_cost = cost
    return lowest_cost, best_a


def distortion(chosen_alternative, profile):
    alternatives = get_all_alternatives(profile)
    if chosen_alternative not in alternatives:
        raise KeyError("The chosen alternative is present in the provided profile")
    profile_with_costs = add_linear_cost(profile)
    
    best_possible_cost, best_a = get_lowest_cost(alternatives, profile_with_costs)
    chosen_alternative_cost = get_total_cost_for_alternative(chosen_alternative, profile_with_costs)
    print(f"Chosen a: {chosen_alternative}({chosen_alternative_cost})\nBest a: {best_a}({best_possible_cost})")
    return chosen_alternative_cost / best_possible_cost

def best_scf(profile):
    """ Work in progress, no idea what I wanted to do here"""
    scf_distortion_pairs = [(scf_name, distortion(scf(profile), profile)) for scf, scf_name in all_scfs_with_names]
    print(list(sorted(scf_distortion_pairs, key=lambda p: p[1])))
    return list(sorted(scf_distortion_pairs, key=lambda p: p[1]))

In [36]:
print(add_linear_cost(profile))
print(dict(add_linear_cost(profile)['Belgium']))
print(distortion('United Kingdom', profile))

{'Belgium': [('Ireland', 0), ('United Kingdom', 1), ('Switzerland', 2), ('Malta', 3), ('Italy', 4), ('Yugoslavia', 5), ('Spain', 6), ('The Netherlands', 7), ('France', 8), ('Israel', 9), ('Finland', 10), ('Germany', 11), ('Luxembourg', 12), ('Monaco', 13), ('Norway', 14), ('Portugal', 15), ('Sweden', 16), ('Turkey', 17)], 'Finland': [('Italy', 0), ('The Netherlands', 1), ('Spain', 2), ('United Kingdom', 3), ('Sweden', 4), ('Luxembourg', 5), ('Switzerland', 6), ('Israel', 7), ('Monaco', 8), ('Ireland', 9), ('Belgium', 10), ('France', 11), ('Germany', 12), ('Malta', 13), ('Norway', 14), ('Portugal', 15), ('Turkey', 16), ('Yugoslavia', 17)], 'France': [('United Kingdom', 0), ('Switzerland', 1), ('Malta', 2), ('Sweden', 3), ('Ireland', 4), ('The Netherlands', 5), ('Italy', 6), ('Luxembourg', 7), ('Portugal', 8), ('Israel', 9), ('Belgium', 10), ('Finland', 11), ('Germany', 12), ('Monaco', 13), ('Norway', 14), ('Spain', 15), ('Turkey', 16), ('Yugoslavia', 17)], 'Germany': [('Finland', 0), ('

In [42]:
best_scf(profile)

Chosen a: The Netherlands(43)
Best a: The Netherlands(43)
Chosen a: Luxembourg(113)
Best a: The Netherlands(43)
Chosen a: The Netherlands(43)
Best a: The Netherlands(43)
Chosen a: The Netherlands(43)
Best a: The Netherlands(43)
Chosen a: The Netherlands(43)
Best a: The Netherlands(43)
[('random_dictatorship', 1.0), ('plurality', 1.0), ('borda', 1.0), ('song_festival_rules', 1.0), ('stv_with_random_dictatorship', 2.627906976744186)]


[('random_dictatorship', 1.0),
 ('plurality', 1.0),
 ('borda', 1.0),
 ('song_festival_rules', 1.0),
 ('stv_with_random_dictatorship', 2.627906976744186)]